In [307]:
import docx
import numpy as np

# Step 1
An article about the history of Washington University is given as the information needs
to be transmitted. This WU article is in English characters, and your group will convert
the contents to binary digits using the ASCII encoding. That is, use the ASCII table and
translate this article to a digital file containing several hundreds of 1’s and 0’s.

In [308]:
# Load the document
doc = docx.Document('text.docx')
message = []
for para in doc.paragraphs:
    message.append(para.text)
text = '\n'.join(message)

# convert to ASCII
bin_digits = ''
for char in text:
    # handling special character
    if char == chr(8212):
        char = chr(45)
    # attach 8 digits for each char
    bin_digits += '0'* (8-len(bin(ord(char))[2:]))
    bin_digits += bin(ord(char))[2:]
    

`bin_digits` is the result after converting the text into binary digits

# Step 2
These ASCII encoded bits in the WU article are now divided into multiple segments, and
each segment can contain a maximum of 200 bits. The last segment may not have all 200
bits. Then, a CRC will be calculated for each segment. The CRC function to be used is
g(D) = [D16 + D12 + D5 + 1], which will generate a CRC length of 16 bits for each
segment. By attaching the 16 CRC bits, a segment becomes a frame and each frame has
216 bits (which include 200 bits of segmented data and 16 bits of CRC). (Note: this is a
traditional Layer 2 function.)

In [309]:
def get_CRC(string):
    string = list(string)
    if len(string) != 200:
        return 'incorrect input length'
    
    # initialize the dividend
    dividend = []
    for i in range(len(string)-1, -1, -1):
        if string[i] =='1':
            dividend.append(i)
    
    divisor = [16, 12, 5, 1]
    
    remainder = [199]
    while remainder[0] >= 16:
        quotient = dividend[0] - divisor[0]
        result = np.add(divisor, quotient)
        
        # get the remainder
        remainder = [x for x in dividend if x not in result] + [x for x in result if x not in dividend]
        remainder.sort(reverse=True)
        
        dividend = remainder
    
    # convert the remainder into 16bits
    CRC = []
    for i in range(15, -1, -1):
        if i in remainder:
            CRC.append('1')
        else:
            CRC.append('0')
    
    CRC = ''.join(CRC)
    return CRC

            
    

In [310]:
i = 0
new_bin_digits = ''
while 200*i < len(bin_digits):
    
    # get the CRC
    test_string = bin_digits[200*i:(200*(i+1))]
    CRC = get_CRC(test_string)

    # attach CRC at the end of each 200 digits
    new_bin_digits += test_string
    new_bin_digits += CRC
    i += 1

`new_bin_digits` is the result after segementing and attaching CRC